# Streamlit

>[Streamlit](https://docs.streamlit.io/) 是一个开源的 Python 库，可以轻松创建和共享精美的、定制化的 Web 应用，用于机器学习和数据科学。

本教程将介绍如何在 `Streamlit` 应用中存储和使用聊天消息历史。`StreamlitChatMessageHistory` 将在指定的 `key=` 下将消息存储在
[Streamlit session state](https://docs.streamlit.io/library/api-reference/session-state) 中。默认的 key 是 `"langchain_messages"`。

- 注意，`StreamlitChatMessageHistory` 仅在 Streamlit 应用中运行时有效。
- 您可能还对 LangChain 的 [StreamlitCallbackHandler](/docs/integrations/callbacks/streamlit) 感兴趣。
- 有关 Streamlit 的更多信息，请查看其
[入门文档](https://docs.streamlit.io/library/get-started)。

该集成位于 `langchain-community` 包中，因此我们需要安装它。我们还需要安装 `streamlit`。

```
pip install -U langchain-community streamlit
```

您可以在此处查看[完整的应用示例运行](https://langchain-st-memory.streamlit.app/)，并在
[github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent) 中找到更多示例。

In [ ]:
from langchain_community.chat_message_histories import (
    StreamlitChatMessageHistory,
)

history = StreamlitChatMessageHistory(key="chat_messages")

history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [ ]:
history.messages

我们可以轻松地将此消息历史记录类与 [LCEL Runnables](/docs/how_to/message_history) 结合使用。

在给定用户会话中，历史记录将在 Streamlit 应用的重新运行时得以保留。给定的 `StreamlitChatMessageHistory` 将不会在用户会话之间保留或共享。

In [ ]:
# Optionally, specify your own session_state key for storing messages
msgs = StreamlitChatMessageHistory(key="special_app_key")

if len(msgs.messages) == 0:
    msgs.add_ai_message("How can I help you?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: msgs,  # Always return the instance created earlier
    input_messages_key="question",
    history_messages_key="history",
)

诸如 "Conversational Streamlit apps" 这类应用，在每次重新运行时，通常会重绘之前的每一条聊天消息。通过遍历 `StreamlitChatMessageHistory.messages` 可以轻松实现这一点：

In [ ]:
import streamlit as st

for msg in msgs.messages:
    st.chat_message(msg.type).write(msg.content)

if prompt := st.chat_input():
    st.chat_message("human").write(prompt)

    # As usual, new messages are added to StreamlitChatMessageHistory when the Chain is called.
    config = {"configurable": {"session_id": "any"}}
    response = chain_with_history.invoke({"question": prompt}, config)
    st.chat_message("ai").write(response.content)

**[查看最终应用](https://langchain-st-memory.streamlit.app/)。**